In [ ]:
import http.server
import socketserver
import cgi
import joblib
import numpy as np

# Load the trained Decision Tree model and label encoder
model = joblib.load('recommendation_model.pkl')
label_encoder = joblib.load('label_encoder.pkl')

class RecommendationHandler(http.server.SimpleHTTPRequestHandler):
    def do_POST(self):
        # Parse the form data
        content_type, pdict = cgi.parse_header(self.headers['Content-Type'])

        if content_type == 'application/x-www-form-urlencoded':
            try:
                length = int(self.headers['Content-Length'])
                post_data = self.rfile.read(length)
                params = cgi.parse_qs(post_data.decode())

                # Extract feature values
                try:
                    login_frequency = float(params.get('login_frequency')[0])
                    time_spent_modules = float(params.get('time_spent_modules')[0])
                    participation_forums = float(params.get('participation_forums')[0])
                    quiz_performance_average = float(params.get('quiz_performance_average')[0])
                    assignment_submissions = float(params.get('assignment_submissions')[0])
                except (TypeError, ValueError, KeyError) as e:
                    self.send_response(400)
                    self.send_header('Content-type', 'text/html')
                    self.end_headers()
                    self.wfile.write(b"<h1>Error: Invalid input data</h1>")
                    return

                # Prepare data for prediction
                features = np.array([[login_frequency, time_spent_modules, participation_forums, 
                                      quiz_performance_average, assignment_submissions]])

                # Predict the engagement level
                try:
                    prediction_encoded = model.predict(features)[0]
                    engagement_level = label_encoder.inverse_transform([prediction_encoded])[0]
                except Exception as e:
                    self.send_response(500)
                    self.send_header('Content-type', 'text/html')
                    self.end_headers()
                    self.wfile.write(b"<h1>Error: Prediction failed</h1>")
                    return

                # Respond with the predicted engagement level
                self.send_response(200)
                self.send_header('Content-type', 'text/html')
                self.end_headers()
                self.wfile.write(f"<h1>Predicted Engagement Level: {engagement_level}</h1>".encode())

            except Exception as e:
                self.send_response(500)
                self.send_header('Content-type', 'text/html')
                self.end_headers()
                self.wfile.write(b"<h1>Error: Server encountered an unexpected error</h1>")
        else:
            self.send_response(400)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            self.wfile.write(b"<h1>Error: Unsupported Content-Type</h1>")

# Start the server
PORT = 8000
Handler = RecommendationHandler

with socketserver.TCPServer(("", PORT), Handler) as httpd:
    print(f"Server running on port {PORT}")
    httpd.serve_forever()
